In [1]:
import tweepy
from tweepy import OAuthHandler
import nltk
from flask import Flask
from flask_cors import CORS
from requests_oauthlib import OAuth1

nltk.download('wordnet')
import pandas as pd
from sklearn.externals import joblib
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chettykulkarni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/chettykulkarni/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/chettykulkarni/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chettykulkarni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
string='ram'
loaded_model = joblib.load('lr_Down.pkl')
vector=joblib.load('abc.pkl')
consumer_key = "veNJxpsprtmX4qvysWCw3DHNG"
consumer_secret = "rnbojQvAPe0guXUg15XMNb4TW7P9xeu9Yp2JvLEwQ452TUpx8q"
access_key = "2422714549-zxSwCa4KHEM3n0Elve7fmkYyFDd5hMexMPRirud"
access_secret = "W4RCAftVTF9kDLknXfC0Tsvh5M44mBrMrlWC1fLznHY5f"
def authorize_twitter_api():
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    return tweepy.API(auth)
def fetch_tweets(keyword, no_of_tweets=10):
    return twitter_api.search(keyword, count=no_of_tweets)
twitter_api = authorize_twitter_api()
keyword = string.split(" ")
no_of_tweets = 100
tweets = []
for i in keyword:
    tweets += [tweet._json for tweet in fetch_tweets(i, no_of_tweets)]
data = pd.DataFrame(tweets)
# data.to_csv('tweet_custom.csv')

df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
df['cleaned_specials'] = df['stemmed'].str.replace('[^A-z0-9 ]', '')
df['cleaned_specials'] = df['stemmed'].str.replace('(\s+)(a|an|and|the)(\s+)', '')
df=df.drop(columns=['lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
vector2 = vector.transform(df['Tweet'])
df['Predicted_Class']=loaded_model.predict(vector2)
# df.to_csv('tweet_custom.csv')
data=df


df = df[df['Predicted_Class'] == 1]
words=' '.join([word for word in df.Tweet if type(word)==str])
x=words.split(" ")
counterWords=Counter(x)
# print(counterWords)
x=[]
value=[]
terminator=0
for i,j in counterWords.items():
    terminator+=1
    if terminator==100:
        break
    x.append(i)
    value.append(j)
df=pd.DataFrame(x)
df=df.rename(columns={0: "x"})
df['value']=value
df = df.sort_values(by=['value'],ascending=False)
df.to_csv('customneg.csv')


df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
df['cleaned_specials'] = df['stemmed'].str.replace('[^A-z0-9 ]', '')
df['cleaned_specials'] = df['stemmed'].str.replace('(\s+)(a|an|and|the)(\s+)', '')
df=df.drop(columns=['lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
vector2 = vector.transform(df['Tweet'])
df['Predicted_Class']=loaded_model.predict(vector2)
# df.to_csv('tweet_custom.csv')
data=df


df = df[df['Predicted_Class'] == 0]
words=' '.join([word for word in df.Tweet if type(word)==str])
x=words.split(" ")
counterWords=Counter(x)
# print(counterWords)
x=[]
value=[]
terminator=0
for i,j in counterWords.items():
    terminator+=1
    if terminator==100:
        break
    x.append(i)
    value.append(j)
df=pd.DataFrame(x)
df=df.rename(columns={0: "x"})
df['value']=value
df = df.sort_values(by=['value'],ascending=False)
df.to_csv('custompos.csv')

df=data
tweets_df=df
tweets_df['Predicted_Class']=df['Predicted_Class']
tweets_df['date']=pd.to_datetime(tweets_df['created_at']).dt.date
tweets_df['hour']=pd.to_datetime(tweets_df['created_at']).dt.hour
tweets_df['min']=pd.to_datetime(tweets_df['created_at']).dt.minute
df=tweets_df
df1=df.groupby(['created_at','Predicted_Class']).count().reset_index()[['created_at','Predicted_Class','Tweet','date','hour','min']]
df2=df1
df=df[['date','hour','min','Predicted_Class','Tweet',]]
df1=df.groupby(['date','hour','min','Predicted_Class']).count().reset_index()[['date','hour','min','Predicted_Class','Tweet']]
df2=df1
result = pd.merge(left=df1,right=df2, how='left', left_on=['date','hour','min'], right_on=['date','hour','min'])
date=result[result.Tweet_x!=result.Tweet_y][['date','hour','min','Tweet_x','Tweet_y']]
date['Tweet_y']=-date['Tweet_y']
date=date.sort_values(['date','hour'],ascending=False)
date=date.drop_duplicates(subset =['date','hour','min'])
date.to_csv('customwater.csv')


return df.Predicted_Class.value_counts()

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


SyntaxError: 'return' outside function (<ipython-input-43-e0e18486ac84>, line 122)

In [53]:
customneg=pd.read_csv('customneg.csv')
customneg.drop('Unnamed: 0',axis=1)

,x,value
0,rt,51
1,ram,18
2,dalit,7
3,के,6
4,है,5
5,को,4
6,में,3
7,이,3
8,@narendramodi,3
9,@kuvalayamala:,3


In [54]:
custompos=pd.read_csv('custompos.csv')
custompos.drop('Unnamed: 0',axis=1)

,x,value
0,rt,11
1,select,8
2,you,8
3,economy.,8
4,ram,8
5,-,6
6,i,5
7,b…,4
8,fix,4
9,sink,4


In [55]:
customwater=pd.read_csv('customwater.csv')
customwater.drop('Unnamed: 0',axis=1)

,date,hour,min,Tweet_x,Tweet_y
0,2019-12-04,12,32,3,-24
1,2019-12-04,12,33,6,-31
2,2019-12-04,12,34,9,-27


In [59]:
import json
a=[1],[2],[3]
print(json.dumps(a))

[[1], [2], [3]]


In [69]:
string='abc'
loaded_model = joblib.load('lr_Down.pkl')
vector=joblib.load('abc.pkl')
consumer_key = "veNJxpsprtmX4qvysWCw3DHNG"
consumer_secret = "rnbojQvAPe0guXUg15XMNb4TW7P9xeu9Yp2JvLEwQ452TUpx8q"
access_key = "2422714549-zxSwCa4KHEM3n0Elve7fmkYyFDd5hMexMPRirud"
access_secret = "W4RCAftVTF9kDLknXfC0Tsvh5M44mBrMrlWC1fLznHY5f"
def authorize_twitter_api():
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    return tweepy.API(auth)
def fetch_tweets(keyword, no_of_tweets=10):
    return twitter_api.search(keyword, count=no_of_tweets)
twitter_api = authorize_twitter_api()
keyword = string.split(" ")
no_of_tweets = 100
tweets = []
for i in keyword:
    tweets += [tweet._json for tweet in fetch_tweets(i, no_of_tweets)]
data = pd.DataFrame(tweets)
# data.to_csv('tweet_custom.csv')

df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
df['cleaned_specials'] = df['stemmed'].str.replace('[^A-z0-9 ]', '')
df['cleaned_specials'] = df['stemmed'].str.replace('(\s+)(a|an|and|the)(\s+)', '')
df=df.drop(columns=['lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
vector2 = vector.transform(df['Tweet'])
df['Predicted_Class']=loaded_model.predict(vector2)
# df.to_csv('tweet_custom.csv')
data=df


df = df[df['Predicted_Class'] == 1]
words=' '.join([word for word in df.Tweet if type(word)==str])
x=words.split(" ")
counterWords=Counter(x)
# print(counterWords)
x=[]
value=[]
terminator=0
for i,j in counterWords.items():
    terminator+=1
    if terminator==100:
        break
    x.append(i)
    value.append(j)
df=pd.DataFrame(x)
df=df.rename(columns={0: "x"})
df['value']=value
df = df.sort_values(by=['value'],ascending=False)
# df.to_csv('customneg.csv')
customneg=df.to_json(orient='records')


df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
df['cleaned_specials'] = df['stemmed'].str.replace('[^A-z0-9 ]', '')
df['cleaned_specials'] = df['stemmed'].str.replace('(\s+)(a|an|and|the)(\s+)', '')
df=df.drop(columns=['lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
vector2 = vector.transform(df['Tweet'])
df['Predicted_Class']=loaded_model.predict(vector2)
# df.to_csv('tweet_custom.csv')
data=df


df = df[df['Predicted_Class'] == 0]
words=' '.join([word for word in df.Tweet if type(word)==str])
x=words.split(" ")
counterWords=Counter(x)
# print(counterWords)
x=[]
value=[]
terminator=0
for i,j in counterWords.items():
    terminator+=1
    if terminator==100:
        break
    x.append(i)
    value.append(j)
df=pd.DataFrame(x)
df=df.rename(columns={0: "x"})
df['value']=value
df = df.sort_values(by=['value'],ascending=False)
# df.to_csv('custompos.csv')
custompos=df.to_json(orient='records')

df=data
tweets_df=df
tweets_df['Predicted_Class']=df['Predicted_Class']
tweets_df['date']=pd.to_datetime(tweets_df['created_at']).dt.date
tweets_df['hour']=pd.to_datetime(tweets_df['created_at']).dt.hour
tweets_df['min']=pd.to_datetime(tweets_df['created_at']).dt.minute
df=tweets_df
df1=df.groupby(['created_at','Predicted_Class']).count().reset_index()[['created_at','Predicted_Class','Tweet','date','hour','min']]
df2=df1
df=df[['date','hour','min','Predicted_Class','Tweet',]]
df1=df.groupby(['date','hour','min','Predicted_Class']).count().reset_index()[['date','hour','min','Predicted_Class','Tweet']]
df2=df1
result = pd.merge(left=df1,right=df2, how='left', left_on=['date','hour','min'], right_on=['date','hour','min'])
date=result[result.Tweet_x!=result.Tweet_y][['date','hour','min','Tweet_x','Tweet_y']]
date['Tweet_y']=-date['Tweet_y']
date=date.sort_values(['date','hour'],ascending=False)
date=date.drop_duplicates(subset =['date','hour','min'])
customwater=date.to_json(orient='records')
item=[]
item.append(str('\''+df.Predicted_Class.value_counts().to_json(orient='records')+'\''))
item.append(customwater)
item.append(custompos)
item.append(customneg)
item2=pd.DataFrame(item)

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [70]:
item2

,0
0,"'[74,26]'"
1,"[{""date"":1575417600000,""hour"":13,""min"":50,""Twe..."
2,"[{""x"":""rt"",""value"":12},{""x"":""@abc"",""value"":5},..."
3,"[{""x"":""rt"",""value"":47},{""x"":""de"",""value"":27},{..."


In [63]:
df.Predicted_Class.value_counts().to_json(orient='records')

'[76,24]'

In [107]:
loaded_model = joblib.load('lr_Down.pkl')
vector=joblib.load('abc.pkl')
consumer_key = "veNJxpsprtmX4qvysWCw3DHNG"
consumer_secret = "rnbojQvAPe0guXUg15XMNb4TW7P9xeu9Yp2JvLEwQ452TUpx8q"
access_key = "2422714549-zxSwCa4KHEM3n0Elve7fmkYyFDd5hMexMPRirud"
access_secret = "W4RCAftVTF9kDLknXfC0Tsvh5M44mBrMrlWC1fLznHY5f"
def authorize_twitter_api():
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    return tweepy.API(auth)
def fetch_tweets(keyword, no_of_tweets=10):
    return twitter_api.search(keyword, count=no_of_tweets)
twitter_api = authorize_twitter_api()
keyword = string.split(" ")
no_of_tweets = 100
tweets = []
for i in keyword:
    tweets += [tweet._json for tweet in fetch_tweets(i, no_of_tweets)]
data = pd.DataFrame(tweets)
# data.to_csv('tweet_custom.csv')

df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 

df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([re.sub('[\W_]+', '', y) for y in x.split()]))
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
regex_pat = re.compile(r'/[^a-zA-Z0-9]+/',flags=re.IGNORECASE)
df['cleaned_specials'] = df['stemmed'].str.replace(regex_pat, '', regex = True)
regex_pat = re.compile(r'(\s+)(a|an|and|the)(\s+)',flags=re.IGNORECASE)
df['cleaned_specials'] = df['stemmed'].str.replace(regex_pat, '', regex = True)



/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [108]:
df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,retweeted,retweeted_status,source,text,truncated,user,Tweet,lemm,stemmed,cleaned_specials
0,None,None,Wed Dec 04 15:08:07 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243188389502976,1202243188389502976,...,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Y’all I’ve been watching abc family Christmas ...,False,"{'id': 888844978293747712, 'id_str': '88884497...",Yall Ive been watching abc family Christmas mo...,Yall Ive watching abc family Christmas movie l...,yall ive watch abc famili christma movi like j...,yall ive watch abc famili christma movi like j...
1,None,None,Wed Dec 04 15:08:07 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1202243108307701761, 'id_str...",0,False,None,1202243187089252353,1202243187089252353,...,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@ABC https://t.co/YAuHnCH9mL,False,"{'id': 770839244621221888, 'id_str': '77083924...",ABC httpstcoYAuHnCH9mL,ABC httpstcoYAuHnCH9mL,abc httpstcoyauhnch9ml,abc httpstcoyauhnch9ml
2,None,None,Wed Dec 04 15:08:06 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243183029161985,1202243183029161985,...,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@suzanneewright @AnthemRespect @Sambyhere @ABC...,False,"{'id': 940044047757983744, 'id_str': '94004404...",suzanneewright AnthemRespect Sambyhere ABC CBS...,suzanneewright AnthemRespect Sambyhere ABC CBS...,suzanneewright anthemrespect sambyher abc cbsn...,suzanneewright anthemrespect sambyher abc cbsn...
3,None,None,Wed Dec 04 15:08:05 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243182160994304,1202243182160994304,...,False,{'created_at': 'Tue Dec 03 20:15:35 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @WordswithSteph: BREAKING: Rep. Adam Schiff...,False,"{'id': 977543461, 'id_str': '977543461', 'name...",RT WordswithSteph BREAKING Rep Adam Schiff on ...,RT WordswithSteph BREAKING Rep Adam Schiff Tru...,rt wordswithsteph break rep adam schiff trump ...,rt wordswithsteph break rep adam schiff trump ...
4,None,None,Wed Dec 04 15:08:05 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243180369981440,1202243180369981440,...,False,{'created_at': 'Wed Dec 04 13:32:53 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @abcdesevilla: Greta Thunberg: La Asociació...,False,"{'id': 4854244389, 'id_str': '4854244389', 'na...",RT abcdesevilla Greta Thunberg La Asociación d...,RT abcdesevilla Greta Thunberg La Asociación d...,rt abcdesevilla greta thunberg la asociación d...,rt abcdesevilla greta thunberg la asociación d...
5,None,None,Wed Dec 04 15:08:04 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243178092470272,1202243178092470272,...,False,{'created_at': 'Wed Dec 04 14:14:09 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @RepMattGaetz: COMING UP: I will be on @ABC...,False,"{'id': 817828029162463237, 'id_str': '81782802...",RT RepMattGaetz COMING UP I will be on ABC to ...,RT RepMattGaetz COMING UP I ABC discus today i...,rt repmattgaetz come up i abc discus today imp...,rt repmattgaetz come up i abc discus today imp...
6,None,None,Wed Dec 04 15:08:03 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202243172245626882,1202243172245626882,...,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@abc_es A los fachas sin escrúpulos no les gus...,True,"{'id': 1169957204528680960, 'id_str': '1169957...",abces A los fachas sin escrúpulos no les gusta...,abces A los fachas sin escrúpulos le gusta que...,abc a los facha sin escrúpulo le gusta que una...,abclos facha sin escrúpulo le gusta que una ni...
7,None,None,Wed Dec 04 15:08:03 +0000 2019,"{'ha

In [74]:
df=df.drop(columns=['lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
vector2 = vector.transform(df['Tweet'])
df['Predicted_Class']=loaded_model.predict(vector2)
# df.to_csv('tweet_custom.csv')
data=df



KeyError: "['lemm' 'stemmed'] not found in axis"

In [73]:
data

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user,Tweet,Predicted_Class
0,None,None,Wed Dec 04 14:48:14 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1202233095434506240, 'id_str...",0,False,None,1202238186476494850,1202238186476494850,...,NaN,305,False,{'created_at': 'Wed Dec 04 14:28:01 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...","RT @Snoopy: Tune-in to ""A Charlie Brown Christ...",False,"{'id': 38430879, 'id_str': '38430879', 'name':...","rt @snoopy: tune-in ""a charli brown christmas""...",1
1,None,None,Wed Dec 04 14:48:14 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238185155239936,1202238185155239936,...,NaN,88,False,{'created_at': 'Wed Dec 04 13:49:37 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @JOSEMANUELSOTO1: Procesan al ex secretario...,False,"{'id': 225146127, 'id_str': '225146127', 'name...",rt @josemanuelsoto1: procesan al ex secretario...,1
2,None,None,Wed Dec 04 14:48:14 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238184576487424,1202238184576487424,...,NaN,4,False,{'created_at': 'Wed Dec 04 04:06:36 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @jennaemoran: So @RCSDNYS has waiting lists...,False,"{'id': 271575588, 'id_str': '271575588', 'name...",rt @jennaemoran: so @rcsdnys ha wait list neig...,0
3,None,None,Wed Dec 04 14:48:13 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238182886166528,1202238182886166528,...,NaN,15,False,{'created_at': 'Wed Dec 04 09:31:40 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @agrarogo_55: La vida y obra de un gran mar...,False,"{'id': 2342636640, 'id_str': '2342636640', 'na...",rt @agrarogo_55: la vida obra de un gran marin...,1
4,None,None,Wed Dec 04 14:48:13 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238179283275777,1202238179283275777,...,NaN,49,False,{'created_at': 'Wed Dec 04 11:46:49 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @antonio_rvega: Procesan al ex secretario g...,False,"{'id': 2260400938, 'id_str': '2260400938', 'na...",rt @antonio_rvega: procesan al ex secretario g...,1
5,None,None,Wed Dec 04 14:48:11 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238172916305921,1202238172916305921,...,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@ABC Damn facts and reality always getting in ...,False,"{'id': 169736296, 'id_str': '169736296', 'name...",@abc damn fact realiti alway get way republica...,0
6,None,None,Wed Dec 04 14:48:09 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238164850692096,1202238164850692096,...,NaN,6,False,{'created_at': 'Wed Dec 04 14:12:54 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @Vox_LPA: Una asociación de vecinos de Tala...,False,"{'id': 957919236952911872, 'id_str': '95791923...",rt @vox_lpa: una asociación de vecino de talav...,1
7,None,None,Wed Dec 04 14:48:09 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1201881093881896965, 'id_str...",0,False,None,1202238162459738114,1202238162459738114,...,NaN,177,False,{'created_at': 'Tue Dec 03 15:09:19 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @BNee_i: 占探 https://t.co/d2YgLe7R9i,False,"{'id': 973347095980605440, 'id_str': '97334709...",rt @bnee_i: 占探 https://t.co/d2ygle7r9i,1
8,None,None,Wed Dec 04 14:48:08 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202238160777793536,1202238160777793536,...,1201824023082127360,1626,False,{'created_at': 'Wed Dec 04 05:05:19 +0000 2019...,"<a href=""https://mobile.twitter.com"" rel=

In [80]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     /Users/chettykulkarni/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [81]:
import re, locale

In [86]:
sall = ''.join(chr(i) for i in range(256))

In [87]:
sall

'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0¡¢£¤¥¦§¨©ª«¬\xad®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ'

In [92]:
x=re.sub('[^a-zA-Z0-9]+', '', sall)

In [93]:
x

'0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

In [122]:
loaded_model = joblib.load('lr_Down.pkl')
vector=joblib.load('abc.pkl')
consumer_key = "veNJxpsprtmX4qvysWCw3DHNG"
consumer_secret = "rnbojQvAPe0guXUg15XMNb4TW7P9xeu9Yp2JvLEwQ452TUpx8q"
access_key = "2422714549-zxSwCa4KHEM3n0Elve7fmkYyFDd5hMexMPRirud"
access_secret = "W4RCAftVTF9kDLknXfC0Tsvh5M44mBrMrlWC1fLznHY5f"
def authorize_twitter_api():
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    return tweepy.API(auth)
def fetch_tweets(keyword, no_of_tweets=10):
    return twitter_api.search(keyword, count=no_of_tweets)
twitter_api = authorize_twitter_api()
keyword = string.split(" ")
no_of_tweets = 100
tweets = []
for i in keyword:
    tweets += [tweet._json for tweet in fetch_tweets(i, no_of_tweets)]
data = pd.DataFrame(tweets)
# data.to_csv('tweet_custom.csv')

df=data

df['Tweet']=df.text
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([re.sub('[^A-z]', '', y) for y in x.split()]))
df['lemm'] = df['Tweet'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['lemm'] = df['lemm'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))

In [123]:
df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,retweet_count,retweeted,retweeted_status,source,text,truncated,user,Tweet,lemm,stemmed
0,None,None,Wed Dec 04 15:14:18 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244743788236800,1202244743788236800,...,2,False,{'created_at': 'Wed Dec 04 12:37:46 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @robbulldog: 'Alarm bells should be ringing...,False,"{'id': 612494885, 'id_str': '612494885', 'name...",RT robbulldog Alarm bells should be ringing ov...,RT robbulldog Alarm bell ringing poor student ...,rt robbulldog alarm bell ring poor student tes...
1,None,None,Wed Dec 04 15:14:17 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244741431201792,1202244741431201792,...,563,False,{'created_at': 'Mon Dec 02 23:11:21 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @either_mark: Después de la viñeta contra m...,False,"{'id': 910468539210960896, 'id_str': '91046853...",RT either_mark Despus de la vieta contra maest...,RT either_mark Despus de la vieta contra maest...,rt either_mark despus de la vieta contra maest...
2,None,None,Wed Dec 04 15:14:17 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244740990873602,1202244740990873602,...,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Así se contagia la pasión por la Historia. ¡Br...,False,"{'id': 1977823740, 'id_str': '1977823740', 'na...",As se contagia la pasin por la Historia Bravo ...,As se contagia la pasin por la Historia Bravo ...,as se contagia la pasin por la historia bravo ...
3,None,None,Wed Dec 04 15:14:16 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244735232020481,1202244735232020481,...,1598,False,{'created_at': 'Tue Dec 03 18:39:44 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...","RT @GunnelsWarren: Retweet if you think CNN, M...",False,"{'id': 16124643, 'id_str': '16124643', 'name':...",RT GunnelsWarren Retweet if you think CNN MSNB...,RT GunnelsWarren Retweet think CNN MSNBC ABC N...,rt gunnelswarren retweet think cnn msnbc abc n...
4,None,None,Wed Dec 04 15:14:15 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244734414151680,1202244734414151680,...,414,False,{'created_at': 'Wed Dec 04 11:03:14 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @TeaBoots: President Trump and First Lady a...,False,"{'id': 1065464553901883392, 'id_str': '1065464...",RT TeaBoots President Trump and First Lady are...,RT TeaBoots President Trump First Lady class a...,rt teaboot presid trump first ladi class act r...
5,None,None,Wed Dec 04 15:14:15 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244733789229056,1202244733789229056,...,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@HUMORJMNIETO @abc_es Quien pinta a los cienti...,True,"{'id': 535348318, 'id_str': '535348318', 'name...",HUMORJMNIETO abc_es Quien pinta a los cientifi...,HUMORJMNIETO abc_es Quien pinta los cientifico...,humorjmnieto abc_ quien pinta los cientifico d...
6,None,None,Wed Dec 04 15:14:15 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244731285245954,1202244731285245954,...,1,False,{'created_at': 'Wed Dec 04 15:14:04 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @dbreed714: @cubfansince76 @reallyDonB @ABC...,False,"{'id': 728840052, 'id_str': '728840052', 'name...",RT dbreed cubfansince reallyDonB ABC If ABC is...,RT dbreed cubfansince reallyDonB ABC If ABC go...,rt dbreed cubfansinc reallydonb abc if abc go ...
7,None,None,Wed Dec 04 15:14:14 +0000 2019,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1202244727749431298,1202244727749431298,...,207,False,{'created_at': 'Wed Dec 04 11:55:59 +000